<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/prompts/advanced_prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 高级提示技巧（变量映射，函数）

在这个笔记本中，我们将展示一些高级的提示技巧。这些功能允许您定义更多定制/表达力强的提示，重用现有的提示，并且用更少的代码表达某些操作。

我们将展示以下功能：
1. 部分格式化
2. 提示模板变量映射
3. 提示函数映射


In [ ]:
%pip install llama-index-llms-openai

In [ ]:
from llama_index.core import PromptTemplate
from llama_index.llms.openai import OpenAI

## 1. 部分格式化

部分格式化（`partial_format`）允许您部分格式化提示，填充一些变量，同时将其他变量留待以后填充。

这是一个很方便的函数，这样您就不必一直保留所有必需的提示变量直到`format`，可以在它们到达时部分格式化。

这将创建提示模板的副本。


In [ ]:
qa_prompt_tmpl_str = """\
下面是上下文信息。
---------------------
{context_str}
---------------------
根据上下文信息而非先验知识，回答问题。
请以{tone_name}的风格写出答案。
问题: {query_str}
答案: \
"""

prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)


In [ ]:
partial_prompt_tmpl = prompt_tmpl.partial_format(tone_name="Shakespeare")

In [ ]:
partial_prompt_tmpl.kwargs

{'tone_name': 'Shakespeare'}

In [ ]:
fmt_prompt = partial_prompt_tmpl.format(
    context_str="In this work, we develop and release Llama 2, a collection of pretrained and fine-tuned large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters",
    query_str="How many params does llama 2 have",
)
print(fmt_prompt)

Context information is below.
---------------------
In this work, we develop and release Llama 2, a collection of pretrained and fine-tuned large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters
---------------------
Given the context information and not prior knowledge, answer the query.
Please write the answer in the style of Shakespeare
Query: How many params does llama 2 have
Answer: 


## 2. 提示模板变量映射

模板变量映射允许您指定从“预期”提示键（例如用于响应合成的`context_str`和`query_str`）到实际模板中的键的映射。

这样可以让您重复使用现有的字符串模板，而无需烦人地更改模板变量。


In [ ]:

# 注意：这里请注意我们使用`my_context`和`my_query`作为模板变量

qa_prompt_tmpl_str = """\
下面是上下文信息。
---------------------
{my_context}
---------------------
根据上下文信息和非先验知识，回答查询。
查询：{my_query}
答案：\
"""

template_var_mappings = {"context_str": "my_context", "query_str": "my_query"}

prompt_tmpl = PromptTemplate(
    qa_prompt_tmpl_str, template_var_mappings=template_var_mappings
)


In [ ]:
fmt_prompt = partial_prompt_tmpl.format(
    context_str="In this work, we develop and release Llama 2, a collection of pretrained and fine-tuned large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters",
    query_str="How many params does llama 2 have",
)
print(fmt_prompt)

Context information is below.
---------------------
In this work, we develop and release Llama 2, a collection of pretrained and fine-tuned large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters
---------------------
Given the context information and not prior knowledge, answer the query.
Please write the answer in the style of Shakespeare
Query: How many params does llama 2 have
Answer: 


### 3. 提示函数映射

您还可以将函数作为模板变量传递，而不是固定值。

这允许您在查询时动态注入某些值，这些值取决于其他值。

以下是一些基本示例。我们在我们的RAG提示工程指南中展示了更高级的示例（例如few-shot示例）。


In [ ]:
qa_prompt_tmpl_str = """\
下面是上下文信息。
---------------------
{context_str}
---------------------
根据上下文信息而非先验知识来回答问题。
问题: {query_str}
回答: \
"""


def format_context_fn(**kwargs):
    # 使用项目符号格式化上下文
    context_list = kwargs["context_str"].split("\n\n")
    fmtted_context = "\n\n".join([f"- {c}" for c in context_list])
    return fmtted_context


prompt_tmpl = PromptTemplate(
    qa_prompt_tmpl_str, function_mappings={"context_str": format_context_fn}
)

In [ ]:
context_str = """\
在这项工作中，我们开发并发布了Llama 2，这是一个包含了预训练和微调的大型语言模型（LLMs），规模从70亿到700亿参数不等。

我们的微调LLMs，称为Llama 2-Chat，经过优化以用于对话场景。

我们的模型在我们测试的大多数基准测试中表现优于开源聊天模型，并根据我们的人工评估，对于帮助性和安全性，可能是封闭源模型的合适替代品。
"""

fmt_prompt = prompt_tmpl.format(
    context_str=context_str, query_str="Llama 2有多少参数"
)
print(fmt_prompt)

Context information is below.
---------------------
- In this work, we develop and release Llama 2, a collection of pretrained and fine-tuned large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.

- Our fine-tuned LLMs, called Llama 2-Chat, are optimized for dialogue use cases.

- Our models outperform open-source chat models on most benchmarks we tested, and based on our human evaluations for helpfulness and safety, may be a suitable substitute for closed-source models.

---------------------
Given the context information and not prior knowledge, answer the query.
Query: How many params does llama 2 have
Answer: 
